In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import pystan
import scipy

In [2]:
df = pd.read_excel("/Users/apple/Downloads/userVisits.xlsx")
df.head()

,user_id,total_visits,weeklyVisits,Interval_visits_int
0,1332384,70,"0,0,0,1,2,1,1,0,0,2,1,0,0,2,1,0,0,0,1,1,2,1,0,...","[3, 0, 0, 0, 2, 0, 2, 0, 3, 0, 0, 0, 1, 4, 2, ..."
1,1332636,4,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","[62, 12, 1, 28]"
2,1334543,4,"1,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","[6, 0]"
3,1336195,1,"0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",[113]
4,1339702,16,"1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...","[110, 14, 29, 7, 29, 2, 2, 1, 0, 2, 4, 1, 1, 1..."


In [3]:
df = df.iloc[df.index <= 200]

In [4]:
visits = df['weeklyVisits'].tolist()

In [5]:
visits_int = list(list(int(a) for a in (str(r).split(',')))for r in visits)

In [6]:
len(visits_int)

201

In [7]:
len(visits_int[10])

113

In [15]:
visitsg_code = """
data {
    int<lower=0> J;// number of users
    int<lower=0> G;// number of groups
    int<lower=0> W;// number of weeks
    vector[G] alpha; // dirichlet param
    int<lower=0> K[G];// number of users in a group
    int y[J,W]; // estimated number of visits next week
    //real<lower=0> sigma[J]; // s.e. of visits estimates
}
parameters {
    //real mu;
    //real<lower=0> tau;
    real<lower=0> eta[J];
    real <lower=0, upper=10> etag[G];
    //vector[G] K;
    vector[G] P;
    //real sigma;
}
transformed parameters {
     
     
     
}
model {
    //tau ~ normal(0,1);
    P ~ dirichlet(alpha);
    K ~ multinomial(P);
    for(g in 1:G){
    for (j in 1:K[g]){
    eta[j] ~ gamma(etag[g],2.0);
    }
    }
    for (j in 1:J){
    for(w in 1:W){
    y[j,w] ~ poisson(eta[j]);
    }
 }
}
"""



visitsg_dat = {'J': len(visits_int),
              'W':113,
              'G':5,
              'alpha':[1.0,1.0,1.0,1.0,1.0],
              
              'y': visits_int
               }



sm = pystan.StanModel(model_code=visitsg_code)
fit = sm.sampling(data=visitsg_dat, iter=1000, chains=4)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_17ef42d71cb65a772c92e7114fb2b9c7 NOW.
/Users/apple/anaconda/lib/python3.6/site-packages/pystan/misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):


RuntimeError: Exception: variable does not exist; processing stage=data initialization; variable name=K; base type=int  (in 'unknown file name' at line 7)


In [25]:
fit


Inference for Stan model: anon_model_930aed676ea8ef8ae7af84e8d6574c79.
4 chains, each with iter=1000; warmup=500; thin=1; 
post-warmup draws per chain=500, total post-warmup draws=2000.

             mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
eta[0]       0.56    0.05   0.07   0.46   0.48   0.55   0.62   0.68      2   5.76
eta[1]     1.3e-3  1.2e-3 1.7e-3 5.7e-6 9.5e-5 4.6e-4 2.6e-3 4.6e-3      2    nan
eta[2]     1.4e-5  1.6e-5 2.3e-51.6e-125.0e-10 5.8e-7 2.7e-5 5.7e-5      2    nan
eta[3]    3.0e-308     0.0    0.02.2e-3082.3e-3082.8e-3083.6e-3084.2e-308   2000    nan
eta[4]        0.1    0.01   0.02   0.08   0.09    0.1   0.12   0.13      2  12.26
eta[5]       0.06  7.0e-3 9.9e-3   0.05   0.06   0.06   0.07   0.08      2   9.21
eta[6]       0.01  5.9e-3 8.4e-3 2.9e-3 4.7e-3   0.01   0.02   0.03      2    nan
eta[7]    8.7e-211     0.0    0.01.7e-3082.4e-3084.0e-3081.4e-2104.9e-210   2000    nan
eta[8]       0.39    0.02   0.03   0.35   0.36   0.39   0.41   